In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-31"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
386,2024-08-31,Tailândia Tbl,02:00,Chaophraya Thunder,T-REX,3.49,1.28,167.5,1.83,1.83,-6.5,1.83,1.83,Q7PCLok5,0.286533,0.781250,0.546448,0.546448,0.067783,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,94,55,7.20,3.88,0.000,0.000,0.655223,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
387,2024-08-31,Mundo Amistoso Interclubes,13:00,Aurora Desio (Ita),Basket Crema (Ita),1.67,2.12,152.5,1.86,1.80,-3.5,2.03,1.70,hlxGHEIc,0.598802,0.471698,0.537634,0.555556,0.070501,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.167915,0.023184,0.125118,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
388,2024-08-31,Mundo Amistoso Interclubes,12:00,BC Nokia (Fin),Heidelberg (Ger),1.49,2.50,158.5,1.85,1.85,-5.5,1.95,1.74,M15MKvV6,0.671141,0.400000,0.540541,0.540541,0.071141,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.357984,0.000000,0.080484,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
389,2024-08-31,Mundo Amistoso Interclubes,09:00,Pinar Karsiyaka (Tur),Manisa (Tur),1.36,2.97,164.5,1.80,1.88,-7.5,1.97,1.73,CSLfrz6e,0.735294,0.336700,0.555556,0.531915,0.071994,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.525839,0.030744,0.091733,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
390,2024-08-31,Mundo Amistoso Interclubes,09:00,Inowroclaw (Pol),Poznań (Pol),2.93,1.37,157.5,1.80,1.86,5.5,1.94,1.77,G8bbaX2T,0.341297,0.729927,0.555556,0.537634,0.071224,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.513064,0.023184,0.064802,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
391,2024-08-31,Vietnã Vba,09:30,Saigon Heat,Can Tho Catfish,1.38,2.70,160.5,1.84,1.82,-6.5,1.98,1.70,8j3JwLj4,0.724638,0.370370,0.543478,0.549451,0.095008,121.560,14.656978,0.120574,1.8,1.643168,0.912871,116.48,1.468,0.164225,0.111870,31.0,150.436,52.421245,0.348462,1.2,1.643168,1.369306,136.94,1.938,0.558185,0.288021,-12.0,91,82,1.28,1.67,109.072,0.000,0.457540,0.007728,0.107603,0.49,0.098,3.877551,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
392,2024-08-31,Venezuela Superliga,20:00,Marinos,Cocodrilos,1.66,2.03,164.5,1.85,1.81,-3.5,2.00,1.68,n5E8vLBs,0.602410,0.492611,0.540541,0.552486,0.095020,282.864,221.787707,0.784079,0.6,1.341641,2.236068,151.69,3.226,2.214324,0.686399,-17.0,135.628,36.359763,0.268084,1.8,1.643168,0.912871,135.09,1.610,0.590424,0.366723,10.0,77,79,1.97,1.71,138.110,137.554,0.141805,0.015456,0.122975,-0.64,-0.128,-5.156250,0.690922,0.6,-0.090922,-2.07,-0.414,-2.487923,0.591265,0.7,0.108735
393,2024-08-31,Venezuela Superliga,20:00,Guaiqueries

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
